In [1]:
# Python Version: 3.11.6
# Version:
#    3.0 - Stored in SQL

In [2]:
import math
import random
import datetime
import os
import re
import time
import requests
from itertools import product
from itertools import groupby
from datetime import datetime
from datetime import date
import numpy as np
import pandas as pd
import extract_fin

# Data Collection
import yfinance as yf
from fredapi import Fred

# MultiThreading
import concurrent.futures as cf

import warnings

import subprocess


# Suppress all warnings
warnings.filterwarnings('ignore')

In [3]:
sql_file_path = '/Users/<update>/Documents/Projects/Financial Data Extract/Output/FINANCIAL DATA.db' # Update

In [4]:
start_time = datetime.now().strftime("%Y-%m-%d %H:%M")
update_financials = 'n'
update_ticker_info = 'n'

In [6]:
all_tickers  = extract_fin.load_sql_table('TICKERS_INFO',sql_file_path)
extract_fin.list_tables(sql_file_path)

# Ticker INFO

In [6]:
if update_ticker_info == 'y':
    # ALL_TICKERS file in the Archive folder contains all tickers
    all_tickers  = extract_fin.load_sql_table('TICKERS_ALL',sql_file_path)
    list_tickers_upd_info = all_tickers['Ticker'].to_list()
    list_tickers_upd_info = [item for item in list_tickers_upd_info if not isinstance(item, bool)] # Temp -  there's a boolean element. Amend ALL_Tickers

In [7]:
if update_ticker_info == 'y':
    # The API stops after a certain number of errors
    # Group the list into x groups of 200 elements
    print(f"{len(list_tickers_upd_info)} tickers have been split into {round(len(list_tickers_upd_info)/ 10)} sublists of ~10")

    lst = list_tickers_upd_info  # List with 10 elements
    n = round(len(list_tickers_upd_info)/ 10)  # Number of sublists

    # Split the list into sublists
    sublists = np.array_split(lst, n)

    # Dynamically create variables for each sublist
    for i, sublist in enumerate(sublists):
        # Use globals() to create a new variable with the name 'sublist_i'
        globals()[f'sublist_{i+1}'] = sublist

    # Print lengths of each dynamically created sublist variable
    for i in range(1, n + 1):
        sublist_var = globals().get(f'sublist_{i}', None)
        if sublist_var is not None:
            print(f"Length of Sublist {i}: {len(sublist_var)}")

In [8]:
if update_ticker_info == 'y':
    dropped_ticker=[]
    all_tickers_new = pd.DataFrame()

In [9]:
if update_ticker_info == 'y':
    # Main loop
    sub = 1  # Current Sub list
    max_retries = 2  # Maximum number of retries for each sublist
    
    while sub <= n:
        attempt = 0
        success = False
        
        while attempt < max_retries and not success:
            try:
                for ticker in globals()[f'sublist_{sub}']:
                    temp_info = yf.Ticker(ticker).info
                    temp_info_df = pd.DataFrame([temp_info])
                    temp_info_df = temp_info_df.dropna(how='all')
                    if temp_info_df.empty:
                        dropped_ticker.append(ticker)
                    else:
                        temp_info_df['Ticker'] = ticker
                        all_tickers_new = pd.concat([all_tickers_new, temp_info_df])

                print(f'sublist_{sub} Complete')
                success = True  # Exit retry loop on success
                
            except Exception as e:
                attempt += 1
                print(f"Error encountered in sublist_{sub}: {e}")
                if attempt < max_retries:
                    print(f"Retrying... (Attempt {attempt} of {max_retries})")
                    time.sleep(5)
                else:
                    print(f"Sublist_{sub} failed after {max_retries} attempts.")
            
        sub += 1  # Move to the next sublist after successful completion or max retries

    dropped_ticker = [str(item) for item in dropped_ticker]

In [10]:
if update_ticker_info == 'y':
    all_tickers_new.drop_duplicates(subset = 'Ticker', keep='last', inplace=True)

    date_columns = [col for col in all_tickers_new.columns if 'date' in col.lower()]

    # Convert Unix timestamps in to datetime
    for col in date_columns:
        if all_tickers_new[col].notnull().any():  # Check if the column is not empty
            all_tickers_new[col] = pd.to_datetime(all_tickers_new[col], unit='s', errors='coerce')

    all_tickers_new.rename(columns={'symbol':'ticker'}, inplace=True)

    all_tickers_new = all_tickers_new.sort_values(by = ['quoteType','firstTradeDateEpochUtc'])

    all_tickers_new.drop(columns='ticker',inplace=True)

In [11]:
if update_ticker_info == 'y':
    dropped_ticker_df = pd.DataFrame(dropped_ticker,columns=['Ticker'])
    dropped_ticker_df

In [12]:
if update_ticker_info == 'y':
    # Using ExcelWriter to write to different sheets
    extract_fin.update_db('TICKERS_ALL',all_tickers_new,sql_file_path)
    extract_fin.update_db('TICKERS_DROPPED',all_tickers_new,sql_file_path)
    all_tickers = all_tickers_new

# EQUITY

In [13]:
folder_path = "/Users/<update>/Documents/Projects/Financial Data Extract/Output/EQUITY"

equity_tickers = all_tickers[all_tickers['quoteType'] == 'EQUITY']

## Prices

In [14]:
curr_equity_prices = extract_fin.load_sql_table('EQUITY_PRICES', sql_file_path)

In [15]:
#  Previously downloaded tickers
equity_prices_down_tickers = equity_tickers[['Ticker']].drop_duplicates()

ticker_start_date= curr_equity_prices[['Ticker','Date']].drop_duplicates(subset = 'Ticker', keep = 'last')
ticker_start_date = dict(zip(ticker_start_date['Ticker'], (ticker_start_date['Date'] - pd.Timedelta(days=5)).dt.strftime('%Y-%m-%d')))

In [16]:
new_equity_prices = pd.DataFrame()

# Fetch historical data
for stock in list(equity_tickers['Ticker']):
    
    try:
        
        if equity_prices_down_tickers['Ticker'].str.contains(stock).any():
            temp = yf.download([stock],start = ticker_start_date[stock])
            temp['Ticker'] = stock
            temp.reset_index(inplace=True)
            new_equity_prices = pd.concat([new_equity_prices,temp])  
            
        # If Ticker is new, then download all available days    
        else:   
            temp = yf.download([stock], period='max')
            temp['Ticker'] = stock
            temp.reset_index(inplace=True)
            new_equity_prices = pd.concat([new_equity_prices,temp])
                 
    except:
        pass

updated_equity_prices = pd.concat([curr_equity_prices,new_equity_prices])
updated_equity_prices.drop_duplicates(subset=['Ticker', 'Date'], keep='last', inplace=True)
updated_equity_prices.sort_values(by = ['Ticker','Date'], inplace = True)
updated_equity_prices['Date'] = pd.to_datetime(updated_equity_prices['Date'], format='mixed')
extract_fin.update_db('EQUITY_PRICES',updated_equity_prices,sql_file_path)
updated_equity_prices.tail(10)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

$EAI: possibly delisted; No price data found  (1d 2017-03-08 -> 2024-08-22)



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

$LGCY: possibly delisted; No price data found  (1d 2019-10-06 -> 2024-08-22)



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

('A', '1999-11-18 00:00:00', 32.54649353027344, 35.765380859375, 28.612302780151367, 31.473533630371097, 26.695695877075195, 62546380.0)
('A', '1999-11-19 00:00:00', 30.713518142700195, 30.75822639465332, 28.47818374633789, 28.880544662475582, 24.496328353881836, 15234146.0)
('A', '1999-11-22 00:00:00', 29.55114364624023, 31.473533630371097, 28.65700912475586, 31.473533630371097, 26.695695877075195, 6577870.0)
('A', '1999-11-23 00:00:00', 30.400571823120117, 31.205293655395508, 28.612302780151367, 28.612302780151367, 24.268814086914062, 5975611.0)
('A', '1999-11-24 00:00:00', 28.701717376708984, 29.998212814331055, 28.612302780151367, 29.372318267822266, 24.91345596313477, 4843231.0)


,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
23273894,ZYXI,2024-08-09,7.76,7.78,7.49,7.590,7.590,147900.0
23273895,ZYXI,2024-08-12,7.58,7.65,7.32,7.380,7.380,100200.0
23273896,ZYXI,2024-08-13,7.45,7.50,7.38,7.440,7.440,88100.0
0,ZYXI,2024-08-14,7.50,7.54,7.15,7.190,7.190,60300.0
1,ZYXI,2024-08-15,7.38,7.66,7.33,7.450,7.450,148200.0
2,ZYXI,2024-08-16,7.42,7.85,7.36,7.760,7.760,370400.0
3,ZYXI,2024-08-19,7.74,7.87,7.63,7.780,7.780,79600.0
4,ZYXI,2024-08-20,7.81,7.81,7.51,7.650,7.650,80500.0
5,ZYXI,2024-08-21,7.67,7.83,7.60,7.730,7.730,53300.0
6,ZYXI,2024-08-22,7.80,7.85,7.70,7.755,7.755,16681.0


## Financial Statements

### Balance Sheet

In [17]:
if update_financials == 'y':
    curr_equity_quart_bs = extract_fin.load_sql_table('EQUITY_BS', sql_file_path)

In [18]:
if update_financials == 'y':

    new_equity_quart_bs = pd.DataFrame()

    for stock in list(equity_tickers['Ticker']):
        try:
        # Fetch historical data
            temp_ticker = yf.Ticker(stock)
            temp = temp_ticker.quarterly_balance_sheet
            temp = temp.T
            temp.index.name = 'Date'
            temp.reset_index(inplace=True)
            temp['Ticker'] = stock
            new_quart_bs = pd.concat([new_equity_quart_bs,temp])
        except:
            pass

    updated_equity_quart_bs = pd.concat([curr_equity_quart_bs,new_equity_quart_bs]) 
    updated_equity_quart_bs.drop_duplicates(subset=['Ticker', 'Date'], keep='last', inplace=True)   
    updated_equity_quart_bs.sort_values(by = ['Ticker','Date'], inplace = True)
    updated_equity_quart_bs['Date'] = pd.to_datetime(updated_equity_quart_bs['Date'], format='mixed')
    extract_fin.update_db('EQUITY_BS',updated_equity_quart_bs,sql_file_path)
    updated_equity_quart_bs.tail(10)

('A', '2022-10-31 00:00:00', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 36000000.0, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, -1148000000.0, 2248000000.0, None, None, None, 934000000.0, 0.0, None, None, None, None, None, None, None, None, None, None, None, None, 0.0, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 1255000000.0, 59000000.0, None, None, None, None, None, None, 36000000.0, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
('A', '2023-01-31 00:00:00', None, None, None, None, None, None, 

### Income Statement

In [19]:
if update_financials == 'y':
    curr_equity_quart_is = extract_fin.load_sql_table('EQUITY_IS', sql_file_path)

In [20]:
if update_financials == 'y':
    new_equity_quart_is = pd.DataFrame()

    for stock in list(equity_tickers['Ticker']):
        try:
        # Fetch historical data
            temp_ticker = yf.Ticker(stock)
            temp = temp_ticker.quarterly_balance_sheet
            temp = temp.T
            temp.index.name = 'Date'
            temp.reset_index(inplace=True)
            temp['Ticker'] = stock
            new_quart_bs = pd.concat([new_equity_quart_is,temp])
        except:
            pass

    updated_equity_quart_is = pd.concat([curr_equity_quart_is,new_equity_quart_is]) 
    updated_equity_quart_is.drop_duplicates(subset=['Ticker', 'Date'], keep='last', inplace=True)   
    updated_equity_quart_is.sort_values(by = ['Ticker','Date'], inplace = True)
    updated_equity_quart_is['Date'] = pd.to_datetime(updated_equity_quart_is['Date'], format='mixed')
    extract_fin.update_db('EQUITY_IS',updated_equity_quart_is,sql_file_path)
    updated_equity_quart_is.tail(10)

('A', '2022-10-31 00:00:00', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 2000000.0, 2000000.0, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
('A', '2023-01-31 00:00:00', 0.0, 0.141, 502000000.0, 352000000.0, 67000000.0, 788000000.0, 502000000.0, 435000000.0, -16000000.0, 9000000.0, 352000000.0, 352000000.0, 1330000000.0, 426000000.0, 297000000.0, 296000000.0, 1.19, 1.19, 352000000.0, 352000000.0, 352000000.0, 352000000.0, 352000000.0, 58000000.0, 410000000.0, None, None, -16000000.0, 9000000.0, 426000000.0, 542000000.0, 123000000.0, 419000000.0, None, None, None, 968000000.0, 788000000.0, 1756000000.0, 1756000000.0, None, None, 2500

### Cash Flow Statement

In [21]:
if update_financials == 'y':
    curr_equity_quart_cf = extract_fin.load_sql_table('EQUITY_CF', sql_file_path)

In [22]:
if update_financials == 'y':
    
    new_equity_quart_cf = pd.DataFrame()

    for stock in list(equity_tickers['Ticker']):
        try:
        # Fetch historical data
            temp_ticker = yf.Ticker(stock)
            temp = temp_ticker.quarterly_cashflow
            temp = temp.T
            temp.index.name = 'Date'
            temp['Date'] = pd.to_datetime(temp['Date'])
            temp.reset_index(inplace=True)
            temp['Ticker'] = stock
            new_quart_bs = pd.concat([new_equity_quart_cf,temp])
        except:
            pass

    updated_equity_quart_cf = pd.concat([curr_equity_quart_cf,new_equity_quart_cf]) 
    updated_equity_quart_cf.drop_duplicates(subset=['Ticker', 'Date'], keep='last', inplace=True)   
    updated_equity_quart_cf.sort_values(by = ['Ticker','Date'], inplace = True)
    updated_equity_quart_cf['Date'] = pd.to_datetime(updated_equity_quart_cf['Date'], format='mixed')
    extract_fin.update_db('EQUITY_CF',updated_equity_quart_cf,sql_file_path)
    updated_equity_quart_cf.tail(10)

('A', '2022-10-31 00:00:00', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 0.0, 0.0, None, None, None, None, None, None, 0.0, None, None, None, None, None, 0.0, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, -500000000.0, 355000000.0, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
('A', '2023-01-31 00:00:00', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, N

# INDEX

In [23]:
folder_path = "/Users/<update>/Documents/Projects/Financial Data Extract/Output/INDEX"

index_tickers = all_tickers[all_tickers['quoteType'] == 'INDEX']

In [24]:
curr_index_prices = extract_fin.load_sql_table('INDICES', sql_file_path)

In [25]:
#  Previously downloaded tickers
index_prices_down_tickers = index_tickers[['Ticker']].drop_duplicates()

ticker_start_date= curr_index_prices[['Ticker','Date']].drop_duplicates(subset = 'Ticker', keep = 'last')
ticker_start_date = dict(zip(ticker_start_date['Ticker'], (ticker_start_date['Date'] - pd.Timedelta(days=5)).dt.strftime('%Y-%m-%d')))

In [26]:
new_index_prices = pd.DataFrame()

# Fetch historical data
for stock in list(index_tickers['Ticker']):
    
    try:
        
        if index_prices_down_tickers['Ticker'].str.contains(stock).any():
            temp = yf.download([stock],start = ticker_start_date[stock])
            temp['Ticker'] = stock
            temp.reset_index(inplace=True)
            new_index_prices = pd.concat([new_index_prices,temp])  
            
        # If Ticker is new, then download all available days    
        else:   
            temp = yf.download([stock], period='max')
            temp['Ticker'] = stock
            temp.reset_index(inplace=True)
            new_index_prices = pd.concat([new_index_prices,temp])
                 
    except:
        pass

updated_index_prices = pd.concat([curr_index_prices,new_index_prices])
updated_index_prices.drop_duplicates(subset=['Ticker', 'Date'], keep='last', inplace=True)
updated_index_prices.sort_values(by = ['Ticker','Date'], inplace = True)
updated_index_prices['Date'] = pd.to_datetime(updated_index_prices['Date'], format='mixed')
extract_fin.update_db('INDICES',updated_index_prices,sql_file_path)
updated_index_prices.tail(10)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

('1997-07-02 00:00:00', 1255.9090576171875, 1261.571044921875, 1147.3310546875, 1199.06103515625, 1199.06103515625, 0.0, '000001.SS')
('1997-07-03 00:00:00', 1194.676025390625, 1194.676025390625, 1149.93994140625, 1150.623046875, 1150.623046875, 0.0, '000001.SS')
('1997-07-04 00:00:00', 1138.9210205078125, 1163.2490234375, 1124.7760009765625, 1159.342041015625, 1159.342041015625, 0.0, '000001.SS')
('1997-07-07 00:00:00', 1161.70703125, 1163.447021484375, 1085.572021484375, 1096.8189697265625, 1096.8189697265625, 0.0, '000001.SS')
('1997-07-08 00:00:00', 1092.7989501953125, 1115.4329833984375, 1066.0439453125, 1109.666015625, 1109.666015625, 0.0, '000001.SS')


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
7203,2024-08-09,4940.870117,4965.770020,4895.700195,4964.310059,4964.310059,0.0,^XAX
7204,2024-08-12,4964.310059,5106.870117,4964.310059,5083.470215,5083.470215,0.0,^XAX
7205,2024-08-13,5083.470215,5110.950195,5055.629883,5107.990234,5107.990234,0.0,^XAX
7206,2024-08-14,5107.990234,5133.140137,5094.470215,5126.959961,5126.959961,0.0,^XAX
7207,2024-08-15,5126.959961,5223.160156,5126.959961,5193.080078,5193.080078,0.0,^XAX
7208,2024-08-16,5193.080078,5235.540039,5178.740234,5217.040039,5217.040039,0.0,^XAX
7209,2024-08-19,5217.040039,5245.100098,5205.540039,5211.379883,5211.379883,0.0,^XAX
7210,2024-08-20,5211.379883,5211.890137,5119.479980,5138.810059,5138.810059,0.0,^XAX
7211,2024-08-21,5138.810059,5199.279785,5129.620117,5176.279785,5176.279785,0.0,^XAX
7212,2024-08-22,5176.279785,5193.189941,5157.100098,5164.310059,5164.310059,0.0,^XAX


# FUNDS

In [27]:
folder_path = "/Users/<update>/Documents/Projects/Financial Data Extract/Output/FUNDS"

funds_tickers = all_tickers[all_tickers['quoteType'].isin(['MUTUALFUND','ETF'])]

In [28]:
curr_funds_prices = extract_fin.load_sql_table('FUNDS', sql_file_path)

In [29]:
#  Previously downloaded tickers
funds_prices_down_tickers = index_tickers[['Ticker']].drop_duplicates()

ticker_start_date= curr_funds_prices[['Ticker','Date']].drop_duplicates(subset = 'Ticker', keep = 'last')
ticker_start_date = dict(zip(ticker_start_date['Ticker'], (ticker_start_date['Date'] - pd.Timedelta(days=5)).dt.strftime('%Y-%m-%d')))

In [30]:
new_funds_prices = pd.DataFrame()

# Fetch historical data
for stock in list(funds_tickers['Ticker']):
    
    try:
        
        if index_prices_down_tickers['Ticker'].str.contains(stock).any():
            temp = yf.download([stock], period='max')
            temp['Ticker'] = stock
            temp.reset_index(inplace=True)
            new_funds_prices = pd.concat([new_funds_prices,temp])  
            
        # If Ticker is new, then download all available days    
        else:   
            temp = yf.download([stock], period='max')
            temp['Ticker'] = stock
            temp.reset_index(inplace=True)
            new_funds_prices = pd.concat([new_funds_prices,temp])
                 
    except:
        pass

updated_funds_prices = pd.concat([curr_funds_prices,new_funds_prices])
updated_funds_prices.drop_duplicates(subset=['Ticker', 'Date'], keep='last', inplace=True)
updated_funds_prices.sort_values(by = ['Ticker','Date'], inplace = True)
updated_funds_prices['Date'] = pd.to_datetime(updated_funds_prices['Date'], format='mixed')
extract_fin.update_db('FUNDS',updated_funds_prices,sql_file_path)
updated_funds_prices.tail(10)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
[' MCX']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
[' SPY']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
[' EWJ']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
[' EWU']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of

$MATF: possibly delisted; No price data found  (1d 1925-09-17 -> 2024-08-23)


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
[' MATF']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['FNCF']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 1925-09-17 -> 2024-08-23)')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['HCRF']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 1925-09-17 -> 2024-08-23)')


$FNCF: possibly delisted; No price data found  (1d 1925-09-17 -> 2024-08-23)
$HCRF: possibly delisted; No price data found  (1d 1925-09-17 -> 2024-08-23)


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
[' FNCF']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
[' HCRF']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
[' WALA']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['XIVH']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 1925-09-17 -> 2024-08-23)')


...

[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['YZC']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 1925-09-17 -> 2024-08-23)')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ZFC']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 1925-09-17 -> 2024-08-23)')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 1925-09-17 -> 2024-08-23)')


$YZC: possibly delisted; No price data found  (1d 1925-09-17 -> 2024-08-23)
$ZFC: possibly delisted; No price data found  (1d 1925-09-17 -> 2024-08-23)
$BF.B: possibly delisted; No price data found  (1d 1925-09-17 -> 2024-08-23)


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
[' ACW']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
[' GRO']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
[' AGU']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
[' AHS']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
[' AF']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
[' AV']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


('2015-11-11 00:00:00', 18.5, 25.899999618530273, 18.0, 24.5, 24.5, 1584639.0, 'AAAP')
('2015-11-12 00:00:00', 24.25, 27.1200008392334, 22.5, 25.0, 25.0, 83061.0, 'AAAP')
('2015-11-13 00:00:00', 25.469999313354492, 26.200000762939453, 24.549999237060547, 25.260000228881836, 25.260000228881836, 67396.0, 'AAAP')
('2015-11-16 00:00:00', 25.010000228881836, 26.190000534057617, 24.13010025024414, 25.020000457763672, 25.020000457763672, 16959.0, 'AAAP')
('2015-11-17 00:00:00', 24.459999084472656, 25.510000228881836, 24.3799991607666, 24.6200008392334, 24.6200008392334, 25972.0, 'AAAP')


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
151,2024-08-09,23.667801,23.667801,23.667801,23.667801,23.667801,35.0,ZZZ
152,2024-08-12,23.469299,23.469299,23.469299,23.469299,23.469299,91.0,ZZZ
153,2024-08-13,23.973801,23.973801,23.973801,23.973801,23.973801,145.0,ZZZ
154,2024-08-14,24.000000,24.000000,23.760000,23.760000,23.760000,113.0,ZZZ
155,2024-08-15,24.209999,24.209999,23.879999,23.879999,23.879999,290.0,ZZZ
156,2024-08-16,24.275499,24.275499,24.275499,24.275499,24.275499,12.0,ZZZ
157,2024-08-19,24.362400,24.362400,24.362400,24.362400,24.362400,61.0,ZZZ
158,2024-08-20,24.398800,24.398800,24.398800,24.398800,24.398800,162.0,ZZZ
159,2024-08-21,24.682800,24.682800,24.682800,24.682800,24.682800,17.0,ZZZ
160,2024-08-22,24.354700,24.354700,24.354700,24.354700,24.354700,33.0,ZZZ


# CURRENCY

## Fiat Currency

In [31]:
folder_path = "/Users/<update>/Documents/Projects/Financial Data Extract/Output/CURRENCY"

ccy_tickers = all_tickers[all_tickers['quoteType'] == 'CURRENCY']

In [32]:
curr_fiat_currency_rates = extract_fin.load_sql_table('CURRENCY_FIAT', sql_file_path)

In [33]:
#  Previously downloaded tickers
ccy_rates_down_tickers = curr_fiat_currency_rates[['Ticker']].drop_duplicates()

ticker_start_date= curr_fiat_currency_rates[['Ticker','Date']].drop_duplicates(subset = 'Ticker', keep = 'last')
ticker_start_date = dict(zip(ticker_start_date['Ticker'], (ticker_start_date['Date'] - pd.Timedelta(days=5)).dt.strftime('%Y-%m-%d')))

In [34]:
new_fiat_currency_rates  = pd.DataFrame()

# Fetch historical data
for stock in list(ccy_tickers['Ticker']):
    
    try:
        
        if ccy_rates_down_tickers['Ticker'].str.contains(stock).any():
            temp = yf.download([stock],start = ticker_start_date[stock])
            temp['Ticker'] = stock
            temp.reset_index(inplace=True)
            new_fiat_currency_rates = pd.concat([new_fiat_currency_rates,temp])  
            
        # If Ticker is new, then download all available days    
        else:   
            temp = yf.download([stock], period='max')
            temp['Ticker'] = stock
            temp.reset_index(inplace=True)
            new_fiat_currency_rates = pd.concat([new_fiat_currency_rates,temp])
                 
    except:
        pass

updated_fiat_currency_rates = pd.concat([curr_fiat_currency_rates,new_fiat_currency_rates])
updated_fiat_currency_rates.drop_duplicates(subset=['Ticker', 'Date'], keep='last', inplace=True)
updated_fiat_currency_rates.sort_values(by = ['Ticker','Date'], inplace = True)
updated_fiat_currency_rates['Date'] = pd.to_datetime(updated_fiat_currency_rates['Date'], format='mixed')
extract_fin.update_db('CURRENCY_FIAT',updated_fiat_currency_rates,sql_file_path)
updated_fiat_currency_rates.tail(10)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

('CAD=X', '2003-09-17 00:00:00', 1.3667000532150269, 1.3703999519348145, 1.3636000156402588, 1.3667000532150269, 1.3667000532150269, 0.0)
('CAD=X', '2003-09-18 00:00:00', 1.366000056266785, 1.368499994277954, 1.3596999645233154, 1.3615000247955322, 1.3615000247955322, 0.0)
('CAD=X', '2003-09-19 00:00:00', 1.3619999885559082, 1.3630000352859497, 1.3457000255584717, 1.3472000360488892, 1.3472000360488892, 0.0)
('CAD=X', '2003-09-22 00:00:00', 1.3413000106811523, 1.355299949645996, 1.33650004863739, 1.354200005531311, 1.354200005531311, 0.0)
('CAD=X', '2003-09-23 00:00:00', 1.3549000024795532, 1.357699990272522, 1.3458000421524048, 1.3551000356674194, 1.3551000356674194, 0.0)


,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
154459,USDSGD=X,2024-08-13,1.32381,1.32473,1.31960,1.32381,1.32381,0.0
154460,USDSGD=X,2024-08-14,1.31710,1.31803,1.31380,1.31710,1.31710,0.0
0,USDSGD=X,2024-08-15,1.31673,1.32295,1.31650,1.31673,1.31673,0.0
1,USDSGD=X,2024-08-16,1.32224,1.32229,1.31730,1.32224,1.32224,0.0
154463,USDSGD=X,2024-08-18,1.31530,1.31823,1.31499,1.31581,1.31581,0.0
2,USDSGD=X,2024-08-19,1.31576,1.31579,1.30840,1.31576,1.31576,0.0
3,USDSGD=X,2024-08-20,1.30788,1.31010,1.30540,1.30788,1.30788,0.0
4,USDSGD=X,2024-08-21,1.30453,1.30837,1.30384,1.30453,1.30453,0.0
5,USDSGD=X,2024-08-22,1.30531,1.31089,1.30450,1.30531,1.30531,0.0
6,USDSGD=X,2024-08-23,1.31009,1.31095,1.30686,1.30921,1.30921,0.0


## Crypto

In [35]:
crypt_ccy_tickers = all_tickers[all_tickers['quoteType'] == 'CRYPTOCURRENCY']

In [36]:
curr_crypt_currency_rates = extract_fin.load_sql_table('CURRENCY_CRYPTO', sql_file_path)

In [37]:
#  Previously downloaded tickers
crypt_ccy_rates_down_tickers = curr_crypt_currency_rates[['Ticker']].drop_duplicates()

ticker_start_date= curr_crypt_currency_rates[['Ticker','Date']].drop_duplicates(subset = 'Ticker', keep = 'last')
ticker_start_date = dict(zip(ticker_start_date['Ticker'], (ticker_start_date['Date'] - pd.Timedelta(days=5)).dt.strftime('%Y-%m-%d')))

In [38]:
new_crypt_currency_rates  = pd.DataFrame()

# Fetch historical data
for stock in list(crypt_ccy_tickers['Ticker']):
    
    try:
        
        if crypt_ccy_rates_down_tickers['Ticker'].str.contains(stock).any():
            temp = yf.download([stock],start = ticker_start_date[stock])
            temp['Ticker'] = stock
            temp.reset_index(inplace=True)
            new_crypt_currency_rates = pd.concat([new_crypt_currency_rates,temp])  
            
        # If Ticker is new, then download all available days    
        else:   
            temp = yf.download([stock], period='max')
            temp['Ticker'] = stock
            temp.reset_index(inplace=True)
            new_crypt_currency_rates = pd.concat([new_crypt_currency_rates,temp])
                 
    except:
        pass

updated_crypt_currency_rates = pd.concat([curr_crypt_currency_rates,new_crypt_currency_rates])
updated_crypt_currency_rates.drop_duplicates(subset=['Ticker', 'Date'], keep='last', inplace=True)
updated_crypt_currency_rates.sort_values(by = ['Ticker','Date'], inplace = True)
updated_crypt_currency_rates['Date'] = pd.to_datetime(updated_crypt_currency_rates['Date'], format='mixed')
extract_fin.update_db('CURRENCY_CRYPTO',updated_crypt_currency_rates,sql_file_path)
updated_crypt_currency_rates.tail(10)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

('2017-11-11 00:00:00', 0.0230570007115602, 0.0254299994558095, 0.0220219995826482, 0.0235249996185302, 0.0235249996185302, 4743394, 'ADA-EUR')
('2017-11-12 00:00:00', 0.0235619992017745, 0.0239659994840621, 0.019370000809431, 0.0205630008131265, 0.0205630008131265, 6243782, 'ADA-EUR')
('2017-11-13 00:00:00', 0.0208950005471706, 0.0225499998778104, 0.0201719999313354, 0.0221280008554458, 0.0221280008554458, 3789282, 'ADA-EUR')
('2017-11-14 00:00:00', 0.0221190005540847, 0.0227150004357099, 0.0216080006211996, 0.0222500003874301, 0.0222500003874301, 2572983, 'ADA-EUR')
('2017-11-15 00:00:00', 0.0221529994159936, 0.02352200075984, 0.0214200001209974, 0.0224450007081031, 0.0224450007081031, 5821423, 'ADA-EUR')


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
162046,2024-08-14,0.576408,0.581889,0.566226,0.569192,0.569192,1138569808,XRP-USD
0,2024-08-15,0.569191,0.573873,0.552453,0.560955,0.560955,1163264081,XRP-USD
1,2024-08-16,0.560957,0.570269,0.554828,0.564516,0.564516,1021214646,XRP-USD
2,2024-08-17,0.564514,0.570540,0.562127,0.566319,0.566319,481701399,XRP-USD
3,2024-08-18,0.566324,0.578752,0.563856,0.563875,0.563875,633645481,XRP-USD
4,2024-08-19,0.563874,0.608509,0.562320,0.598606,0.598606,1592288724,XRP-USD
5,2024-08-20,0.598610,0.614521,0.588849,0.594428,0.594428,1275196279,XRP-USD
6,2024-08-21,0.594427,0.605304,0.586758,0.600275,0.600275,1185128571,XRP-USD
7,2024-08-22,0.600277,0.603866,0.593852,0.597601,0.597601,943673179,XRP-USD
8,2024-08-23,0.597611,0.601136,0.596809,0.596809,0.596809,931709760,XRP-USD


# FRED

In [39]:
# Step 1: Set up your FRED API key
FRED_API_KEY = '2745a53615f7e95d6ed45bfb3fa892f3'  # Replace with your actual FRED API key

# Step 2: Initialize the Fred client
fred = Fred(api_key=FRED_API_KEY)

# Step 3: Define the list of series IDs to download
series_list = [
    # GDP Data
    ('US GDP (Nominal)', 'GDP'),
    ('US Real GDP', 'GDPC1'),

    # Inflation Data
    ('US Inflation Rate (CPI-U)', 'CPIAUCNS'),
    ('US Core CPI', 'CPILFESL'),
    ('US Producer Price Index (PPI)', 'PPIACO'),

    # Unemployment Data
    ('US Unemployment Rate', 'UNRATE'),

    # Interest Rates
    ('US Federal Funds Rate', 'FEDFUNDS'),

    # SOFR
    ('SOFR', 'SOFR'),

    # LIBOR Rates
    ('LIBOR 1-Month', 'LIBOR1MO'),
    ('LIBOR 3-Month', 'LIBOR3MO'),
    ('LIBOR 6-Month', 'LIBOR6MO'),
    ('LIBOR 12-Month', 'LIBOR12MO'),

    # Treasury Yields
    ('US 1-Month Treasury Yield', 'DGS1MO'),
    ('US 3-Month Treasury Yield', 'DGS3MO'),
    ('US 6-Month Treasury Yield', 'DGS6MO'),
    ('US 1-Year Treasury Yield', 'DGS1'),
    ('US 2-Year Treasury Yield', 'DGS2'),
    ('US 5-Year Treasury Yield', 'DGS5'),
    ('US 7-Year Treasury Yield', 'DGS7'),
    ('US 10-Year Treasury Yield', 'DGS10'),
    ('US 20-Year Treasury Yield', 'DGS20'),
    ('US 30-Year Treasury Yield', 'DGS30'),

    # Industrial Production
    ('US Industrial Production Index', 'INDPRO'),

    # Retail Sales
    ('US Retail Sales', 'RSXFS'),

    # Housing Market
    ('US Housing Starts', 'HOUST'),
    ('US Building Permits', 'PERMIT'),

    # Trade Data
    ('US Trade Balance (Net Exports)', 'NETEXP'),

    # Recession Indicator
    ('US Recession Indicator', 'RECESSION'),

    # UK Economic Data
    ('UK Bank Rate', 'IRLTLT01GBM156N'),
    ('UK GDP', 'GDPUK'),
    ('UK Inflation Rate', 'UKINF'),
    ('UK Unemployment Rate', 'UKUNEMP'),

]

# Step 4: Fetch and store the data
economic_data = {}

for label, series_id in series_list:
    try:
        # Fetch full historical data without specifying dates
        data = fred.get_series(series_id)
        economic_data[label] = data
    except Exception as e:
        print(f"Could not retrieve data for {label} ({series_id}): {e}")

# Convert the dictionary of Series to a DataFrame
updated_econ = pd.DataFrame(economic_data).reset_index()
updated_econ.rename(columns={'index':'Date'}, inplace=True)
updated_econ

extract_fin.update_db('FRED',updated_econ,sql_file_path)



Could not retrieve data for LIBOR 1-Month (LIBOR1MO): Bad Request.  The series does not exist.
Could not retrieve data for LIBOR 3-Month (LIBOR3MO): Bad Request.  The series does not exist.
Could not retrieve data for LIBOR 6-Month (LIBOR6MO): Bad Request.  The series does not exist.
Could not retrieve data for LIBOR 12-Month (LIBOR12MO): Bad Request.  The series does not exist.
Could not retrieve data for US Recession Indicator (RECESSION): Bad Request.  The series does not exist.
Could not retrieve data for UK GDP (GDPUK): Bad Request.  The series does not exist.
Could not retrieve data for UK Inflation Rate (UKINF): Bad Request.  The series does not exist.
Could not retrieve data for UK Unemployment Rate (UKUNEMP): Bad Request.  The series does not exist.
('1913-01-01 00:00:00', None, None, 9.8, None, 12.1, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
('1913-02-01 00:00:00', None, None, 9.8, None, 12.0, None,

# Notification

In [40]:
end_time = datetime.now().strftime("%Y-%m-%d %H:%M")

def send_imessage(phone_number, message):
    """
    Send an iMessage to a specified phone number.
    
    :param phone_number: The recipient's phone number.
    :param message: The message to send.
    """
    apple_script = f'''
    tell application "Messages"
        set targetService to 1st service whose service type = iMessage
        set targetBuddy to buddy "{phone_number}" of targetService
        send "{message}" to targetBuddy
    end tell
    '''
    
    subprocess.run(['osascript', '-e', apple_script])

if __name__ == "__main__":
    phone_number = "<update>"  # Replace with your phone number
    end_time = datetime.now().strftime("%Y-%m-%d %H:%M")
    message = f'''Finacial Data Updated
        Start: {start_time}
        End: {end_time}
            '''
    
    send_imessage(phone_number, message)
